Este notebook incluye scripts y funciones para realizar tareas como:

- Alineamiento de secuencias de imágenes solares.
- Corrección de oscilaciones debidas a la atmósfera o al telescopio.
- Filtrado de ruido.
- Mejora de contraste y calibración fotométrica.

> ⚠️ **Nota:**  
> Los cubos de datos originales (`.fits`) **no se incluyen** en este archivo debido a su gran tamaño y a restricciones de derechos de autor.  
> No obstante, el código es completamente funcional: basta con **cambiar el nombre del archivo `.fits`** por uno local equivalente para aplicar el mismo procedimiento.

Este cuaderno busca servir como una guía reproducible y adaptable para el preprocesamiento de imágenes solares con fines científicos.


# Construcción del cubo de datos a partir de archivos `.fts` individuales

En esta sección se ensamblan las imágenes solares individuales con extensión `.fts` en un cubo de datos tridimensional.  
Este paso es necesario porque, en este caso particular, las observaciones no se entregaron en un solo archivo `.fits` tipo cubo, sino como imágenes sueltas correspondientes a distintos tiempos.

Cada archivo `.fts` representa una imagen bidimensional del Sol tomada en un instante específico. Al organizarlas secuencialmente según su nombre o timestamp, se construye un arreglo 3D que puede usarse para análisis temporales o para aplicar procesos de corrección espacial y espectral.

In [ ]:
import numpy as np
from astropy.io import fits
import glob
from datetime import datetime

# Obtener todos los archivos .fts del directorio y ordenarlos alfabéticamente (para asegurar secuencia temporal)
path = sorted(glob.glob("/home/oscar/Escritorio/h_alpha/imagenes/*.fts"))

# Inicializar listas vacías para almacenar datos e información temporal
cubo_datos = []
fechas_obs = []

for archivo in path:
    hdul = fits.open(archivo)
    header = hdul[0].header
    
    # Extraer la fecha de observación si está disponible
    if "DATE-OBS" in header:
        fecha_obs_str = header["DATE-OBS"]
        try:
            fecha_obs_datetime = datetime.strptime(fecha_obs_str, '%Y-%m-%dT%H:%M:%S')
            fechas_obs.append(fecha_obs_datetime)
        except ValueError:
            print(f"Formato de fecha no reconocido en {archivo}: {fecha_obs_str}")
    
    # Leer la imagen desde la extensión adecuada (ajusta el índice si es necesario)
    try:
        data = hdul[0].data  # Usa [0] si las imágenes están en la cabecera principal
        cubo_datos.append(data)
    except Exception as e:
        print(f"Error al leer los datos de {archivo}: {e}")
    
    hdul.close()

# Convertir la lista de imágenes en un arreglo 3D
cubo_datos = np.array(cubo_datos)

# Guardar el cubo de datos en un nuevo archivo .fits
fits.writeto("cubo_datos_crudo.fits", cubo_datos, overwrite=True)

# (Opcional) Guardar fechas de observación como archivo de texto
with open("fechas_observacion.txt", "w") as f:
    for fecha in fechas_obs:
        f.write(f"{fecha.isoformat()}\n")


## Extracción y corrección localizada del cubo de datos solares

Este bloque de código está diseñado específicamente para trabajar con un **sector localizado de una mancha solar**, sin necesidad de cargar ni procesar todo el campo de visión del Sol tranquilo (*quiet Sun*). Esto reduce el costo computacional y permite enfocar los análisis en una región de interés científica.

El procedimiento está dividido en cinco segmentos de tiempo que representan distintos intervalos de la observación. Para cada uno, se definen coordenadas espaciales específicas de recorte, y en algunos casos, se aplica una rotación mediante `scipy.ndimage.rotate`. Esta rotación es necesaria porque el plano de la imagen estaba inclinado durante la adquisición, debido a la orientación del telescopio o el instrumento.

> ⚠️ **Importante:**  
> Los valores de los cortes espaciales (`i`, `j`, `lim_x`, `lim_y`) y del ángulo de rotación son **característicos de esta observación** y pueden variar para otros conjuntos de datos.  
> Se recomienda al usuario inspeccionar visualmente algunos frames utilizando `matplotlib.pyplot.imshow()` para asegurarse de que los recortes estén bien centrados y contengan la región de interés deseada.

En resumen, este código permite:
- Cortar el cubo en una región específica que contiene la mancha solar.
- Corregir rotaciones producto de la observación.
- Visualizar frames claves para validar el proceso.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits 
from scipy.ndimage import rotate
import os

# Cargar el cubo de datos previamente generado
cubo_datos_crudo = fits.getdata("cubo_datos_crudo.fits")

# Parámetros de corte espacial
lim_x = (50, 1400)
lim_y = (50, 1100)
size_x = lim_x[1] - lim_x[0]
size_y = lim_y[1] - lim_y[0]

# Coordenadas para la segunda, tercera, cuarta y quinta copia
i, j = 94, 75

# Índices de separación temporal
primer_frame = 132
segundo_frame = 203
tercer_frame = 213
cuarto_frame = 279
quinto_frame = 438

# Ángulo de rotación para la tercera sección
angulo = 20

# ----- Crear subconjuntos del cubo -----

# 1. Primer corte: hasta frame 132, con recorte rectangular fijo
primera_copia = cubo_datos_crudo[0:primer_frame+1, lim_y[0]:lim_y[1], lim_x[0]:lim_x[1]]

# 2. Segundo corte: frames 133 a 203, usando coordenadas (i,j)
segunda_copia = cubo_datos_crudo[primer_frame+1:segundo_frame+1, i:i+size_y, j:j+size_x]

# 3. Tercer corte: frames 204 a 213, igual que el anterior pero rotado
tercera_copia = rotate(
    cubo_datos_crudo[segundo_frame+1:tercer_frame+1, i:i+size_y, j:j+size_x],
    angle=angulo, reshape=False, order=0
)

# 4. Cuarto corte: frames 214 a 279, sin rotación
cuarta_copia = cubo_datos_crudo[tercer_frame+1:cuarto_frame+1, i:i+size_y, j:j+size_x]

# 5. Quinto corte: frames 280 en adelante
quinta_copia = cubo_datos_crudo[cuarto_frame+1:, i:i+size_y, j:j+size_x]

# ----- (Opcional) Concatenar todas las partes -----
# nuevo_cubo = np.concatenate((primera_copia, segunda_copia, tercera_copia, cuarta_copia, quinta_copia), axis=0)
# fits.writeto('primer_cambio.fits', nuevo_cubo, overwrite=True)

# ----- Visualización -----

# Mostrar y guardar imagen del frame 132 (último de la primera sección)
plt.imshow(cubo_datos_crudo[primer_frame, lim_y[0]:lim_y[1], lim_x[0]:lim_x[1]], cmap="gray")
plt.title("Frame 132")
plt.savefig("imagen_132.png")
plt.show()

# Mostrar y guardar imagen del frame 133 (inicio de la segunda sección)
plt.imshow(cubo_datos_crudo[primer_frame+1, i:i+size_y, j:j+size_x], cmap="gray")
plt.title("Frame 133")
plt.savefig("imagen_133.png")
plt.show()

# Información general
print("Dimensiones del cubo original:", cubo_datos_crudo.shape)

# ----- (Opcional) Guardar todos los frames como PNGs -----
# carpeta_destino = "imagenes_frames_primer_cambio"
# os.makedirs(carpeta_destino, exist_ok=True)
# for idx, frame in enumerate(cubo_datos_crudo):
#     plt.imshow(frame, cmap='gray')
#     plt.title(f"Frame {idx}")
#     plt.savefig(os.path.join(carpeta_destino, f"frame_{idx}.png"))
#     plt.close()

## Corrección de traslación entre imágenes por correlación cruzada en el espacio de Fourier

En esta sección se corrige el desplazamiento (traslación) entre imágenes consecutivas dentro de un cubo de datos solares previamente alineado de forma básica.  
Se utiliza el método `chi2_shift` del paquete `image_registration`, que implementa una técnica basada en la correlación cruzada en el espacio de Fourier, logrando **precisión subpíxel**.

El procedimiento consiste en:

1. Cargar el cubo de datos alineado de forma previa (con el mismo método).
2. Eliminar manualmente ciertos frames corruptos o inconsistentes (indicados por sus índices), en nuestro caso es por no cumplir la resolución optima.
3. Recortar una región central de cada imagen (`lim_x`, `lim_y`) para centrar el análisis en la zona activa de interés.
4. Calcular los desplazamientos entre pares de imágenes consecutivas mediante `chi2_shift(image1, image2)`.
5. Aplicar la corrección al frame siguiente mediante un desplazamiento subpíxel con `scipy.ndimage.shift`.
6. Guardar el cubo corregido como nuevo archivo `.fits`.

> 📌 Este método asume que las imágenes tienen **solo traslaciones** (sin rotación), y que la estructura solar de referencia está contenida en la ventana recortada.

> ⚠️ Asegúrate de que la región `(lim_x, lim_y)` contenga suficiente señal para que la correlación funcione correctamente.

Este enfoque mejora la calidad del alineamiento temporal, esencial para estudios como la detección de movimientos sutiles, análisis de fluctuaciones o seguimiento de estructuras solares finas.


In [ ]:
import numpy as np
from astropy.io import fits
from image_registration import chi2_shift
from scipy.ndimage import shift

# --- Parámetros del recorte espacial donde se hace la comparación ---
lim_y = (325, 800)
lim_x = (400, 1000)

# --- Cargar cubo previamente alineado de forma básica ---
ruta_entrada = "/home/oscar/Escritorio/h_alpha/cross_correlation_interpixel/cubo_alineado.fits"
cubo = fits.getdata(ruta_entrada)

# --- Eliminar manualmente los frames problemáticos ---
indices_malos = [
    2, 346, 347, 352, 353, 371, 372, 373, 391,
    406, 408, 425, 434, 435, 444, 445, 447,
    452, 455, 456, 457, 458, 460, 461, 465, 467
]
cubo = np.delete(cubo, indices_malos, axis=0)

# --- Inicializar el cubo corregido ---
cubo_corregido = cubo.copy()

# --- Corrección de traslación entre imágenes usando FFT ---
for i in range(cubo.shape[0] - 1):
    ref = cubo_corregido[i, lim_y[0]:lim_y[1], lim_x[0]:lim_x[1]]
    mov = cubo[i+1, lim_y[0]:lim_y[1], lim_x[0]:lim_x[1]]
    
    dx, dy, _, _ = chi2_shift(ref, mov)
    cubo_corregido[i+1] = shift(cubo[i+1], shift=(-dy, -dx))  # Nota: orden es (y, x)

    print(f"Frame {i+1}: shift aplicado dx = {dx:.2f}, dy = {dy:.2f}")

# --- Guardar el cubo corregido ---
ruta_salida = "/home/oscar/Escritorio/h_alpha/cross_correlation_interpixel/cubo_alineado_2.fits"
fits.writeto(ruta_salida, cubo_corregido, overwrite=True)